[Node 64: Programme der Python Standard Library](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node64.html)

Navigation:

**Next:** [Verteilte Programme mit XML-RPC](node65.ipynb) **Up:** [Verteilte Programme mit XML-RPC](node65.ipynb) **Previous:** [Verteilte Programme mit XML-RPC](node65.ipynb)

## Python standard library modules

Classic applications such as FTP, sending and receiving e-mails or Telnet can be processed synchronously, i.e. a request is sent to a server, waits for its response and then continues to work. There are modules in the Python standard library for numerous network services, such as ``ftplib, poplib, imaplib, smtplib, nntplib, gopherlib, telnetlib`` .

### Server with SocketServer

We won't go into the [Berkeley-Socket-API](https://en.wikipedia.org/wiki/Berkeley_sockets) here, but limit ourselves to the classes of the ``SocketServer`` module, which hide the complexity of the *low-level* socket API.

With a ``SocketServer`` you proceed as follows:
* Derive processing class from [``BaseRequestHandler``](https://docs.python.org/3/library/socketserver.html#socketserver.BaseRequestHandler) and override ``handle`` method to handle incoming requests ([``BaseRequestHandler``](https://docs.python.org/3/library/socketserver.html#socketserver.BaseRequestHandler)).
* Use server class [``TCPServer``](https://docs.python.org/3/library/socketserver.html#socketserver.TCPServer), [``TCPServer``](https://docs.python.org/3/library/socketserver.html#socketserver.TCPServer) and instantiate it with server address and processing class
* Start server event loop.

An example of a simple TCP-based server:

In [ ]:
from socketserver import TCPServer, BaseRequestHandler

class HelloHandler(BaseRequestHandler):
    def handle(self):
        print ("Serving client:", self.client_address)
        self.request.sendall(b'Hello Client! I am a HelloHandler\r\n')
        
PORT = 7070
TCPServer.allow_reuse_address = True
srv = TCPServer(("", PORT), HelloHandler)
try:
    srv.serve_forever()
    print ("Serving forever at port", PORT)
except:
    print ("Closing the server.")
    srv.server_close()


You run this program on the server and contact the server program from a client on the shell command line with ``nc (netcat)``:

```bash
$ nc localhost 7070
Hello Client! I am a HelloHandler
```

On the server you get a message with this connection:
<pre>
Serving client: ('127.0.0.1', 48908)
</pre>
(see above...)

---
In the following example, we write an echo server that sends back requests from clients unchanged. (Handy, typically for testing.)
To do this, we use the [`StreamRequestHandler`](https://docs.python.org/3/library/socketserver.html#socketserver.StreamRequestHandler) class as the base for our handler, which in turn derives from the `BaseRequestHandler` class used above, but still provides two properties, `self.rfile` and `self.wfile`, about which we get data from the clients receive and send to them. We want to prefix our echo with `"S:"` to identify it as a server response.

#### Digression: Text encoding
In Python 3, strings are encoded in Unicode by default:
> Strings are immutable sequences of Unicode code points. ... The source character set is defined by the encoding declaration; it is [UTF-8](https://de.wikipedia.org/wiki/UTF-8) if no [UTF-8](https://de.wikipedia.org/wiki/UTF-8) is given in the source file

Unicode is an international standard that aims to solve problems caused by incompatible encodings of special characters (such as umlauts) by specifying a digital code for every known character, i.e. a union of all encodings.

Since `line` is of type `bytes` (a sequence of bytes) and not `str`, we need to prefix the literal `"S:"` with a `b` to have it interpreted as `bytes`:

In [ ]:
type("S:")

In [ ]:
type(b"S:")

Byte literals are heavily restricted:
> Bytes literals produce an instance of the bytes type instead of the str type. They may only contain ASCII characters; bytes with a numeric value of 128 or greater must be expressed with escapes.

If we want to convert a string into a byte literal (or vice versa), we use appropriate functions for that:
* [`encode()`](https://docs.python.org/3/library/stdtypes.html#str.encode): `string` $\rightarrow$ `bytes`
* [`decode()`](https://docs.python.org/3/library/stdtypes.html#bytes.decode): `bytes` $\rightarrow$ `string`

In [ ]:
type("S:".encode("UTF-8"))

An inappropriate text encoding leads to the typical image of broken umlauts:

In [ ]:
"Süßes Tiramisù".encode("UTF-8").decode("iso-8859-15")

In [ ]:
"Süßes Tiramisù".encode("iso-8859-15").decode("utf-8", errors="replace")

In [ ]:
"Süßes Tiramisù".encode("UTF-8").decode("UTF-8")

In [ ]:
"Süßes Tiramisù".encode("iso-8859-1").decode("iso-8859-1")

The goal of introducing Unicode, which has already established itself in many operating systems as UTF-8 or UTF-16, is to avoid this.

---

Back to our echo server:

In [ ]:
from socketserver import TCPServer, StreamRequestHandler

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print("Serving client:", self.client_address)
        for line in (self.rfile):
            reply = b"S:" + line
            self.wfile.write(reply)
            
PORT = 7071
TCPServer.allow_reuse_address = True
srv = TCPServer(("", PORT), EchoHandler)
try:
    srv.serve_forever()
    print ("Serving forever at port", PORT)
except:
    print ("Closing the server.")
    srv.server_close()

If you start the above server, you get the following input and output on the client:
<pre>
$nc localhost 7071
Hi
S: Hello
echo !
S: echo !
</pre>

The downside of the Echo Server just introduced lies in the fact that it can only serve one client at a time, since the server is single-threaded.
A solution is to use an asynchronous server (like `Twisted`), or run the ``handle`` method in its own process or thread.

The following example uses ``ThreadingTCPServer`` instead of ``TCPServer``:

In [ ]:
from socketserver import ThreadingTCPServer, StreamRequestHandler

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print("Serving client:", self.client_address)
        for line in (self.rfile):
            reply = b"S:" + line
            self.wfile.write(reply)
            
PORT = 7072
ThreadingTCPServer.allow_reuse_address = True
srv = ThreadingTCPServer(("", PORT), EchoHandler)
try:
    srv.serve_forever()
    print ("Serving forever at port", PORT)
except:
    print ("Closing the server.")
    srv.server_close()

Side note: The ``ThreadingTCPServer`` is an example of the mix-ins mentioned earlier. Its definition is the following line:
```python
class ThreadingTCPServer(ThreadingMixIn, TCPServer): pass

```

### Send emails with `smtplib.SMTP`

To send emails synchronously, you can use the ``SMTP`` class from the ``smtplib`` module. The following program demonstrates how to first compose an e-mail with the ``email.message`` module and then authenticate yourself on the mail server of the LMU physics and send the e-mail using the SMTP protocol (Simple Mail Transfer Protocol ) sent to his own address:
[``test_send_mail.py``](source/test_send_mail.py)

The password is read in via the ``getpass`` function at runtime of the program.

Originally from 1982, SMTP is very simple, but it is still the basis of all mail traffic on the Internet.
It is written in such a way that with a small set of simple commands, a person can interactively compose and send an email in a server connection.
It also provides no authentication. Any server can, in principle, send mail to any other server without having to prove that it is authorized to do so, or that the mail it forwards actually comes from the claimed sender. This quirk in the design of the protocol, while in the spirit of a free and open Internet, also led to the huge spam problem we have today.

### Reading emails with `imaplib.IMAP4`

The ``IMAP4_SSL`` (IMAP = Internet Message Access Protocol) function of the ``imaplib`` module can be used to read emails from a server: [``imaplib_imap4client.py``](source/imaplib_imap4client.py)